# Benchmark Summary
This notebook aggregates runtime measurements for multiple circuit families and highlights backend crossover points.

In [ ]:
from benchmarks.circuits import ghz_circuit, qft_circuit, grover_circuit, random_circuit
from benchmarks.backends import (
    StatevectorAdapter, StimAdapter, MPSAdapter,
    DecisionDiagramAdapter, AerStatevectorAdapter,
    AerMPSAdapter, MQTDDAdapter,
)
from benchmarks.runner import BenchmarkRunner
from quasar import SimulationEngine
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

families = {
    'ghz': ghz_circuit,
    'qft': qft_circuit,
    'grover': grover_circuit,
    'random': lambda n: random_circuit(n, seed=42),
}
qubit_counts = range(2, 13, 2)
backends = [StatevectorAdapter(), StimAdapter(), MPSAdapter(), DecisionDiagramAdapter()]
for cls in [AerStatevectorAdapter, AerMPSAdapter, MQTDDAdapter]:
    try:
        backends.append(cls())
    except Exception:
        pass
runner = BenchmarkRunner()
for name, fn in families.items():
    for n in qubit_counts:
        circ = fn(n)
        for b in backends:
            rec = runner.run(circ, b)
            rec['qubits'] = n
            rec['family'] = name
        rec = runner.run_quasar(circ, SimulationEngine())
        rec['qubits'] = n
        rec['family'] = name
df = pd.DataFrame(runner.results)
sns.relplot(data=df, x='qubits', y='time', hue='framework', col='family', kind='line', facet_kws={'sharey': False})
plt.yscale('log')
plt.show()

# Identify crossover points where the fastest backend changes
def find_crossover(group):
    g = group.sort_values('qubits')
    best = g.loc[g.groupby('qubits')['time'].idxmin()][['qubits','framework']]
    change = best[best['framework'] != best['framework'].shift()].copy()
    return change
crossover = df.groupby('family').apply(find_crossover)
crossover
